In [1]:
from os.path import join
import pandas as pd

import gensim
from gensim.corpora import Dictionary, MmCorpus
from gensim.models import TfidfModel, LdaModel, LdaMulticore
from gensim.models.phrases import Phrases, Phraser

import pyLDAvis as ldavis
import pyLDAvis.gensim
ldavis.enable_notebook()

import logging
#logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

from constants import FULL_PATH, ETL_PATH, NLP_PATH, SMPL_PATH, POS, NOUN, PROPN, TOKEN, HASH, SENT_IDX

def docs_to_lists(token_series):
    return tuple(token_series.tolist())

In [4]:
dataset = ['OnlineParticipation', 'FAZ'][1]
df = pd.read_pickle(join(SMPL_PATH, dataset + '_simple.pickle'))
df = df[df[POS].isin({NOUN, PROPN, 'NER', 'NPHRASE'})]
df = df.groupby([HASH])[TOKEN].agg(docs_to_lists)

In [8]:
# loads a list of random document ids for a balanced dataset (~2000 documents per category)
# for the FAZ dataset
faz_sample = join(ETL_PATH, 'FAZ_document_sample.pickle')
faz_sample = pd.read_pickle(faz_sample)
faz_sample_ids = faz_sample.index
faz_sample_ids

Int64Index([ 4301134201484101152, -5696035671390904143, -6533682718526575818,
            -3346664675579691204, -8358676581287387714, -2785236834914555528,
             4539470495046700910, -6291733065589777869,  3008594611884955597,
             2667660060079641753,
            ...
            -9201876886379869587,  -682332148408733217, -1477403614105270264,
            -6046850756068127617, -8426549411078832312, -4079889429213597579,
            -2812039998456727519, -4001184660305417764,   903203255141181056,
            -3327279407656915164],
           dtype='int64', name='hash', length=13643)

In [9]:
import numpy as np
intersect = np.intersect1d(faz_sample.index.values, df.index.values)
print(intersect)
print(intersect.shape)
union = np.union1d(faz_sample.index.values, df.index.values)
print(union)
print(union.shape)

[-6541785991313574912 -4282370157978565120 -3678630174048332800
 -3595003702759523840 -2779766014965026304 -2307713969445782016
 -2191663350948864256 -2180314715888122624 -2011831270334207232
 -1771118729572549120 -1703077179928423936 -1467507433706368256
 -1465005113624309248 -1122323515148345088  -852078218234603136
  -306863025342409024  -277170277859988320  -157035938325186656
   -89604267562993744   -77822968137134912   -48992859930099920
   -42678520111220440   -33351537468997156   -31115158608833216
   -27818198464873360   -10506512567653280    -8670420270801952
    -8301321570435398    -5791255954927385    -4511041435369750
    -1369466082829884     -294469541870598      263848088864845
      374369825057928     1443054829291776     4040132589990543
     4824689116034861     5286315956173707     7720799316789147
     9270184643551342    14930834443638534    15812556456997382
    16300360208839788    16919702910232478    27226271131462520
    27349755401085380    306790357997635

In [5]:
df

hash
-9222966757875651584    (Bereich, Feingeist, Grobstoffel, Deutschland,...
-9222617066393794560    (Name, Hurrikan, Irma, Harvey, Harvey, Irma, J...
-9222111851211850752    (Serie, Arbeit, Kaffeeküche, Karriere, Hort, K...
-9221822663289604096    (Juppé, Ersatzkandidat, Konservativer, Fillons...
-9221773540373386240    (Facebook-Datenskandal, Peter, Thiels, Firma, ...
-9221136108316593152    (Sex-Skandal, Oxfam, Fall, Sex-Skandal, Oxfam,...
-9220846605603615744    (Arbeitsrecht, VW, Islamist, Vw-Mitarbeiter, A...
-9220723349603270656    (Bundeskanzlerin, Tag, Nachdenken, Angela_Merk...
-9220026754916126720    (Nobelpreisträger-Tagung_Lindau, Intelligenz, ...
-9219891685417034752    (Börsenkorrektur, Besitzer, Aktienfonds, Börse...
-9219492558743169024    (Ausstellung, Adsorbens_Haag, Mondrian, Kunst,...
-9219211593729905664    (Gerichtsbeschluss, terrorverdächtig_Tunesier,...
-9219200106309165056    (Norm, Benziner, Porsche, Abgasnebel, stuttgar...
-9218322072988174336    (Gipfelfi

In [53]:
df_balanced = df[df.index.isin(faz_sample_ids)]
df_balanced.size

74

In [30]:
texts = df.values
dictionary = Dictionary(texts)
dictionary.filter_extremes(no_below=5, no_above=0.5)

# filter some noice (e.g. special characters)
bad_tokens = ['–']
bad_token_ids = [dictionary.token2id[token] for token in bad_tokens]
dictionary.filter_tokens(bad_ids=bad_token_ids, good_ids=None)

bow_corpus = [dictionary.doc2bow(text) for text in texts]
tfidf = TfidfModel(bow_corpus)
tfidf_corpus = tfidf[bow_corpus]

In [31]:
corpus = [bow_corpus, tfidf_corpus][0]
MmCorpus.serialize('../data/{}.mm'.format(dataset), corpus)
corpus_fake = MmCorpus('../data/{}.mm'.format(dataset))

In [19]:
alphas=[0.001, 0.1, 1, 10]
etas=[0.001, 0.1, 1, 10]#[0]

In [20]:
%%time

ldamodel_alphas = []
for alpha in alphas:
    ldamodel_etas = []
    for eta in etas:
        #ldamodel = LdaModel(
        ldamodel = LdaMulticore(
            random_state=42,
            # corpus=bow_corpus, 
            corpus=tfidf_corpus,
            num_topics=10, 
            id2word=dictionary, 
            alpha=alpha,  # 'auto',
            eta=eta,
            # gamma_threshold
            # minimum_phi_value
            # offset
            # decay
            # update_every=1, # not in multicore implementation 
            chunksize=10000, 
            passes=1,
        )
        ldamodel_etas.append(ldamodel)
    ldamodel_alphas.append(ldamodel_etas)
    
#prepared_data = ldavis.gensim.prepare(ldamodel, corpus_fake, dictionary)

CPU times: user 11min 38s, sys: 26.4 s, total: 12min 4s
Wall time: 11min 57s


In [23]:
for a in ldamodel_alphas:
    for e in ldamodel_etas:
        for topic in e.print_topics(num_topics=10, num_words=10):
            print(topic)
        print()
    print()

(0, '0.002*"Prozent" + 0.002*"Trump" + 0.002*"Mann" + 0.002*"Präsident" + 0.001*"Euro" + 0.001*"Mensch" + 0.001*"Frau" + 0.001*"Deutschland" + 0.001*"Kind" + 0.001*"Land"')
(1, '0.002*"Trump" + 0.002*"Prozent" + 0.002*"Präsident" + 0.002*"Unternehmen" + 0.001*"–" + 0.001*"Mensch" + 0.001*"Euro" + 0.001*"Kind" + 0.001*"Million" + 0.001*"Frau"')
(2, '0.002*"Trump" + 0.002*"Euro" + 0.002*"Prozent" + 0.001*"Präsident" + 0.001*"Frau" + 0.001*"Deutschland" + 0.001*"Polizei" + 0.001*"Unternehmen" + 0.001*"–" + 0.001*"Mensch"')
(3, '0.002*"Trump" + 0.002*"Prozent" + 0.001*"Deutschland" + 0.001*"Unternehmen" + 0.001*"Land" + 0.001*"Präsident" + 0.001*"Dollar" + 0.001*"Euro" + 0.001*"Kind" + 0.001*"Polizei"')
(4, '0.002*"Trump" + 0.002*"Prozent" + 0.002*"Deutschland" + 0.002*"Frau" + 0.002*"Präsident" + 0.001*"Euro" + 0.001*"Mensch" + 0.001*"Kind" + 0.001*"Land" + 0.001*"Milliarde"')
(5, '0.003*"Trump" + 0.002*"Prozent" + 0.002*"Euro" + 0.001*"Mann" + 0.001*"–" + 0.001*"Deutschland" + 0.001*"Prä

In [32]:
alpha=0.001
eta=0.001

ldamodel = LdaMulticore(
    random_state=42,
    # corpus=bow_corpus, 
    corpus=tfidf_corpus,
    num_topics=10, 
    id2word=dictionary, 
    alpha=alpha,  # 'auto',
    eta=eta,
    # gamma_threshold
    # minimum_phi_value
    # offset
    # decay
    # update_every=1, # not in multicore implementation 
    chunksize=10000, 
    passes=1,
)

for topic in e.print_topics(num_topics=10, num_words=10):
    print(topic)

prepared_data = ldavis.gensim.prepare(ldamodel, corpus_fake, dictionary)
prepared_data

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
5      12.634855        1       1 -0.009817 -0.012494
1      12.132717        1       2 -0.004478 -0.004875
4      11.201427        1       3 -0.002114  0.016472
0      10.598137        1       4  0.006163  0.026891
7      10.204579        1       5  0.005242  0.011405
9       9.498053        1       6  0.006411  0.019715
3       9.231486        1       7  0.003478  0.007787
8       9.040896        1       8 -0.011227  0.008821
6       8.576749        1       9 -0.042834 -0.035169
2       6.881100        1      10  0.049177 -0.038554, topic_info=      Category          Freq         Term         Total  loglift  logprob
term                                                                     
120    Default  12856.000000        Trump  12856.000000  30.0000  30.0000
866    Default   6511.000000      Polizei   6511.000000  29.0000  29.0000
5807   Default   4386.000000    Nordkorea   4386.000000  28.0000  28.0000
1119   Default   5148.000000     Russland   5148.000000  27.0000  27.0000
548    Default  11448.000000      Prozent  11448.000000  26.0000  26.0000
5207   Default   2567.000000        Putin   2567.000000  25.0000  25.0000
1562   Default   3597.000000      Erdogan   3597.000000  24.0000  24.0000
914    Default   7166.000000         Kind   7166.000000  23.0000  23.0000
110    Default   8809.000000    Präsident   8809.000000  22.0000  22.0000
2759   Default   3108.000000         Iran   3108.000000  21.0000  21.0000
3573   Default   2910.000000     Seehofer   2910.000000  20.0000  20.0000
6805   Default   1792.000000   Bundeswehr   1792.000000  19.0000  19.0000
909    Default   3470.000000        Elter   3470.000000  18.0000  18.0000
1138   Default   2409.000000        Apple   2409.000000  17.0000  17.0000
5161   Default   2502.000000       Google   2502.000000  16.0000  16.0000
198    Default   7446.000000  Unternehmen   7446.000000  15.0000  15.0000
356    Default   9304.000000         Euro   9304.000000  14.0000  14.0000
228    Default    634.000000       Fillon    634.000000  13.0000  13.0000
233    Default   7554.000000         Frau   7554.000000  12.0000  12.0000
24173  Default    263.000000          ZTE    263.000000  11.0000  11.0000
4270   Default    162.000000    Kaspersky    162.000000  10.0000  10.0000
30680  Default    119.000000       Weimer    119.000000   9.0000   9.0000
2623   Default   2619.000000      Wohnung   2619.000000   8.0000   8.0000
147    Default   3898.000000     Facebook   3898.000000   7.0000   7.0000
1550   Default   5062.000000       Türkei   5062.000000   6.0000   6.0000
2074   Default    904.000000         Opel    904.000000   5.0000   5.0000
8952   Default    746.000000        Talib    746.000000   4.0000   4.0000
7264   Default   1137.000000    Jerusalem   1137.000000   3.0000   3.0000
1053   Default   4318.000000       Trumps   4318.000000   2.0000   2.0000
11657  Default    865.000000       Mieter    865.000000   1.0000   1.0000
...        ...           ...          ...           ...      ...      ...
914    Topic10    520.294739         Kind   7166.001465   0.0537  -6.6588
909    Topic10    318.003662        Elter   3470.937988   0.2863  -7.1512
356    Topic10    586.935913         Euro   9304.743164  -0.0870  -6.5383
1119   Topic10    401.452393     Russland   5148.673828   0.1250  -6.9181
446    Topic10    396.794189       Partei   5087.847656   0.1252  -6.9298
13     Topic10    521.147583  Deutschland   8099.197754  -0.0671  -6.6572
305    Topic10    467.027039      Million   7017.390137  -0.0334  -6.7668
147    Topic10    323.824524     Facebook   3898.585938   0.1882  -7.1330
1874   Topic10    326.106110           EU   3997.034912   0.1703  -7.1260
2605   Topic10    385.303894          SPD   5375.964844   0.0407  -6.9592
43     Topic10    459.173798         Mann   7352.069824  -0.0969  -6.7838
1055   Topic10    293.019196         Wahl   3477.3942

In [33]:
alpha=0.1
eta=0.001

ldamodel = LdaMulticore(
    random_state=42,
    # corpus=bow_corpus, 
    corpus=tfidf_corpus,
    num_topics=10, 
    id2word=dictionary, 
    alpha=alpha,  # 'auto',
    eta=eta,
    # gamma_threshold
    # minimum_phi_value
    # offset
    # decay
    # update_every=1, # not in multicore implementation 
    chunksize=10000, 
    passes=1,
)

for topic in e.print_topics(num_topics=10, num_words=10):
    print(topic)

prepared_data = ldavis.gensim.prepare(ldamodel, corpus_fake, dictionary)
prepared_data

(0, '0.000*"Prozent" + 0.000*"Trump" + 0.000*"Mann" + 0.000*"Präsident" + 0.000*"Euro" + 0.000*"Mensch" + 0.000*"Frau" + 0.000*"Deutschland" + 0.000*"Kind" + 0.000*"Land"')
(1, '0.000*"Trump" + 0.000*"Prozent" + 0.000*"Präsident" + 0.000*"Unternehmen" + 0.000*"–" + 0.000*"Mensch" + 0.000*"Euro" + 0.000*"Kind" + 0.000*"Million" + 0.000*"Frau"')
(2, '0.000*"Trump" + 0.000*"Euro" + 0.000*"Prozent" + 0.000*"Präsident" + 0.000*"Frau" + 0.000*"Deutschland" + 0.000*"Polizei" + 0.000*"Unternehmen" + 0.000*"–" + 0.000*"Mensch"')
(3, '0.000*"Trump" + 0.000*"Prozent" + 0.000*"Deutschland" + 0.000*"Unternehmen" + 0.000*"Land" + 0.000*"Präsident" + 0.000*"Dollar" + 0.000*"Euro" + 0.000*"Kind" + 0.000*"Polizei"')
(4, '0.000*"Trump" + 0.000*"Prozent" + 0.000*"Deutschland" + 0.000*"Frau" + 0.000*"Präsident" + 0.000*"Euro" + 0.000*"Mensch" + 0.000*"Kind" + 0.000*"Land" + 0.000*"Milliarde"')
(5, '0.000*"Trump" + 0.000*"Prozent" + 0.000*"Euro" + 0.000*"Mann" + 0.000*"–" + 0.000*"Deutschland" + 0.000*"Prä

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
1      12.085072        1       1 -0.007036 -0.000743
5      12.063912        1       2 -0.024486  0.002976
4      11.862933        1       3  0.005546 -0.010687
0      10.928137        1       4  0.028683 -0.022181
7      10.235408        1       5  0.008043 -0.006046
9       9.808939        1       6  0.015896 -0.015560
8       9.066616        1       7 -0.009171 -0.010425
3       9.024292        1       8  0.010519 -0.005473
6       8.254124        1       9 -0.050293  0.006866
2       6.670571        1      10  0.022300  0.061273, topic_info=     Category          Freq          Term         Total  loglift  logprob
term                                                                     
120   Default  12857.000000         Trump  12857.000000  30.0000  30.0000
5807  Default   4398.000000     Nordkorea   4398.000000  29.0000  29.0000
1119  Default   5126.000000      Russland   5126.000000  28.0000  28.0000
866   Default   6501.000000       Polizei   6501.000000  27.0000  27.0000
1562  Default   3606.000000       Erdogan   3606.000000  26.0000  26.0000
548   Default  11424.000000       Prozent  11424.000000  25.0000  25.0000
110   Default   8800.000000     Präsident   8800.000000  24.0000  24.0000
5207  Default   2556.000000         Putin   2556.000000  23.0000  23.0000
914   Default   7138.000000          Kind   7138.000000  22.0000  22.0000
2759  Default   3094.000000          Iran   3094.000000  21.0000  21.0000
3573  Default   2914.000000      Seehofer   2914.000000  20.0000  20.0000
909   Default   3457.000000         Elter   3457.000000  19.0000  19.0000
1138  Default   2411.000000         Apple   2411.000000  18.0000  18.0000
147   Default   3886.000000      Facebook   3886.000000  17.0000  17.0000
2074  Default    906.000000          Opel    906.000000  16.0000  16.0000
1550  Default   5068.000000        Türkei   5068.000000  15.0000  15.0000
198   Default   7439.000000   Unternehmen   7439.000000  14.0000  14.0000
415   Default   2989.000000        Syrien   2989.000000  13.0000  13.0000
7264  Default   1129.000000     Jerusalem   1129.000000  12.0000  12.0000
1053  Default   4308.000000        Trumps   4308.000000  11.0000  11.0000
853   Default   1791.000000           Kim   1791.000000  10.0000  10.0000
6805  Default   1810.000000    Bundeswehr   1810.000000   9.0000   9.0000
4163  Default   2062.000000      Sanktion   2062.000000   8.0000   8.0000
228   Default    628.000000        Fillon    628.000000   7.0000   7.0000
5161  Default   2505.000000        Google   2505.000000   6.0000   6.0000
2623  Default   2613.000000       Wohnung   2613.000000   5.0000   5.0000
3918  Default   2184.000000        Moskau   2184.000000   4.0000   4.0000
2945  Default   1397.000000         Tesla   1397.000000   3.0000   3.0000
1493  Default   1423.000000        Kirche   1423.000000   2.0000   2.0000
4270  Default    158.000000     Kaspersky    158.000000   1.0000   1.0000
...       ...           ...           ...           ...      ...      ...
198   Topic10    547.143066   Unternehmen   7439.860840   0.0976  -6.5775
147   Topic10    351.893341      Facebook   3886.709961   0.3055  -7.0189
909   Topic10    326.227448         Elter   3457.595703   0.3467  -7.0946
110   Topic10    580.120605     Präsident   8800.025391  -0.0118  -6.5190
1119  Topic10    399.324615      Russland   5126.965332   0.1550  -6.8924
446   Topic10    393.680267        Partei   5086.054199   0.1487  -6.9067
356   Topic10    564.237488          Euro   9323.607422  -0.0974  -6.5467
13    Topic10    502.362366   Deutschland   8109.839844  -0.0740  -6.6629
305   Topic10    459.227875       Million   7030.765625  -0.0210  -6.7527
1874  Topic10    321.587921            EU   3991.285400   0.1889  -7.1089
1055  Topic10    291.867767          Wahl   3472.302979   0.2312  -7.2059
294   Topic10    278.608734         Daten   3212.4357

In [34]:
alpha=1
eta=0.001

ldamodel = LdaMulticore(
    random_state=42,
    # corpus=bow_corpus, 
    corpus=tfidf_corpus,
    num_topics=10, 
    id2word=dictionary, 
    alpha=alpha,  # 'auto',
    eta=eta,
    # gamma_threshold
    # minimum_phi_value
    # offset
    # decay
    # update_every=1, # not in multicore implementation 
    chunksize=10000, 
    passes=1,
)

for topic in e.print_topics(num_topics=10, num_words=10):
    print(topic)

prepared_data = ldavis.gensim.prepare(ldamodel, corpus_fake, dictionary)
prepared_data

(0, '0.000*"Prozent" + 0.000*"Trump" + 0.000*"Mann" + 0.000*"Präsident" + 0.000*"Euro" + 0.000*"Mensch" + 0.000*"Frau" + 0.000*"Deutschland" + 0.000*"Kind" + 0.000*"Land"')
(1, '0.000*"Trump" + 0.000*"Prozent" + 0.000*"Präsident" + 0.000*"Unternehmen" + 0.000*"–" + 0.000*"Mensch" + 0.000*"Euro" + 0.000*"Kind" + 0.000*"Million" + 0.000*"Frau"')
(2, '0.000*"Trump" + 0.000*"Euro" + 0.000*"Prozent" + 0.000*"Präsident" + 0.000*"Frau" + 0.000*"Deutschland" + 0.000*"Polizei" + 0.000*"Unternehmen" + 0.000*"–" + 0.000*"Mensch"')
(3, '0.000*"Trump" + 0.000*"Prozent" + 0.000*"Deutschland" + 0.000*"Unternehmen" + 0.000*"Land" + 0.000*"Präsident" + 0.000*"Dollar" + 0.000*"Euro" + 0.000*"Kind" + 0.000*"Polizei"')
(4, '0.000*"Trump" + 0.000*"Prozent" + 0.000*"Deutschland" + 0.000*"Frau" + 0.000*"Präsident" + 0.000*"Euro" + 0.000*"Mensch" + 0.000*"Kind" + 0.000*"Land" + 0.000*"Milliarde"')
(5, '0.000*"Trump" + 0.000*"Prozent" + 0.000*"Euro" + 0.000*"Mann" + 0.000*"–" + 0.000*"Deutschland" + 0.000*"Prä

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
8      10.465289        1       1  0.000737  0.002291
1      10.454765        1       2  0.002294  0.001025
7      10.258004        1       3  0.001913 -0.002806
4      10.232146        1       4 -0.001072 -0.001920
9      10.161436        1       5  0.002108  0.000152
0      10.101927        1       6 -0.001284 -0.000980
3       9.803179        1       7  0.000056 -0.000926
5       9.747883        1       8 -0.002329 -0.000756
6       9.701774        1       9 -0.002621  0.001702
2       9.073595        1      10  0.000198  0.002217, topic_info=     Category          Freq              Term         Total  loglift  logprob
term                                                                         
110   Default   8792.000000         Präsident   8792.000000  30.0000  30.0000
120   Default  12784.000000             Trump  12784.000000  29.0000  29.0000
548   Default  11434.000000           Prozent  11434.000000  28.0000  28.0000
1119  Default   5128.000000          Russland   5128.000000  27.0000  27.0000
866   Default   6559.000000           Polizei   6559.000000  26.0000  26.0000
305   Default   6991.000000           Million   6991.000000  25.0000  25.0000
885   Default   4525.000000             Stadt   4525.000000  24.0000  24.0000
191   Default   4000.000000              Teil   4000.000000  23.0000  23.0000
897   Default   5173.000000              Zeit   5173.000000  22.0000  22.0000
142   Default   5840.000000            Dollar   5840.000000  21.0000  21.0000
57    Default   5379.000000         Regierung   5379.000000  20.0000  20.0000
118   Default   5393.000000               Tag   5393.000000  19.0000  19.0000
2276  Default   3732.000000              Buch   3732.000000  18.0000  18.0000
294   Default   3216.000000             Daten   3216.000000  17.0000  17.0000
1874  Default   3981.000000                EU   3981.000000  16.0000  16.0000
352   Default   3783.000000          Dienstag   3783.000000  15.0000  15.0000
342   Default   4747.000000  vereinigen_Staat   4747.000000  14.0000  14.0000
13    Default   8098.000000       Deutschland   8098.000000  13.0000  13.0000
1775  Default   3775.000000              Film   3775.000000  12.0000  12.0000
709   Default   2171.000000       Möglichkeit   2171.000000  11.0000  11.0000
464   Default   1906.000000              Arzt   1906.000000  10.0000  10.0000
614   Default   3997.000000              Bild   3997.000000   9.0000   9.0000
415   Default   2956.000000            Syrien   2956.000000   8.0000   8.0000
3603  Default   3013.000000               AfD   3013.000000   7.0000   7.0000
356   Default   9341.000000              Euro   9341.000000   6.0000   6.0000
929   Default   3220.000000               CSU   3220.000000   5.0000   5.0000
945   Default   3172.000000             Vater   3172.000000   4.0000   4.0000
251   Default   2741.000000         Politiker   2741.000000   3.0000   3.0000
778   Default   4372.000000             Monat   4372.000000   2.0000   2.0000
815   Default   3120.000000              Ziel   3120.000000   1.0000   1.0000
...       ...           ...               ...           ...      ...      ...
272   Topic10    172.827759         Verfügung   1378.267822   0.3235  -8.0376
342   Topic10    498.312531  vereinigen_Staat   4747.858398   0.1456  -6.9786
310   Topic10    288.443115            Nutzer   2509.698730   0.2364  -7.5254
651   Topic10    391.218628             Seite   3633.005371   0.1713  -7.2206
363   Topic10    258.856110         Interview   2217.028076   0.2522  -7.6336
1055  Topic10    376.268372              Wahl   3485.176514   0.1738  -7.2596
116   Topic10    351.769379            Studie   3230.790039   0.1823  -7.3269
1379  Topic10    307.511841   Bundesregierung   2740.599121   0.2124  -7.4614
914   Topic10    651.892334              Kind   7209.034180  -0.0034  -6.7100
897   Topic10    504.157135      

In [35]:
alpha=1
eta=0.00001

ldamodel = LdaMulticore(
    random_state=42,
    # corpus=bow_corpus, 
    corpus=tfidf_corpus,
    num_topics=10, 
    id2word=dictionary, 
    alpha=alpha,  # 'auto',
    eta=eta,
    # gamma_threshold
    # minimum_phi_value
    # offset
    # decay
    # update_every=1, # not in multicore implementation 
    chunksize=10000, 
    passes=1,
)

for topic in e.print_topics(num_topics=10, num_words=10):
    print(topic)

prepared_data = ldavis.gensim.prepare(ldamodel, corpus_fake, dictionary)
prepared_data

(0, '0.000*"Prozent" + 0.000*"Trump" + 0.000*"Mann" + 0.000*"Präsident" + 0.000*"Euro" + 0.000*"Mensch" + 0.000*"Frau" + 0.000*"Deutschland" + 0.000*"Kind" + 0.000*"Land"')
(1, '0.000*"Trump" + 0.000*"Prozent" + 0.000*"Präsident" + 0.000*"Unternehmen" + 0.000*"–" + 0.000*"Mensch" + 0.000*"Euro" + 0.000*"Kind" + 0.000*"Million" + 0.000*"Frau"')
(2, '0.000*"Trump" + 0.000*"Euro" + 0.000*"Prozent" + 0.000*"Präsident" + 0.000*"Frau" + 0.000*"Deutschland" + 0.000*"Polizei" + 0.000*"Unternehmen" + 0.000*"–" + 0.000*"Mensch"')
(3, '0.000*"Trump" + 0.000*"Prozent" + 0.000*"Deutschland" + 0.000*"Unternehmen" + 0.000*"Land" + 0.000*"Präsident" + 0.000*"Dollar" + 0.000*"Euro" + 0.000*"Kind" + 0.000*"Polizei"')
(4, '0.000*"Trump" + 0.000*"Prozent" + 0.000*"Deutschland" + 0.000*"Frau" + 0.000*"Präsident" + 0.000*"Euro" + 0.000*"Mensch" + 0.000*"Kind" + 0.000*"Land" + 0.000*"Milliarde"')
(5, '0.000*"Trump" + 0.000*"Prozent" + 0.000*"Euro" + 0.000*"Mann" + 0.000*"–" + 0.000*"Deutschland" + 0.000*"Prä

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
8      10.464784        1       1  0.000739  0.002303
1      10.454314        1       2  0.002303  0.001033
7      10.257168        1       3  0.001923 -0.002817
4      10.227928        1       4 -0.001077 -0.001929
9      10.159236        1       5  0.002119  0.000153
0      10.102451        1       6 -0.001288 -0.000986
3       9.805097        1       7  0.000058 -0.000930
5       9.748106        1       8 -0.002340 -0.000760
6       9.705930        1       9 -0.002634  0.001708
2       9.074982        1      10  0.000198  0.002226, topic_info=     Category          Freq              Term         Total  loglift  logprob
term                                                                         
110   Default   8812.000000         Präsident   8812.000000  30.0000  30.0000
120   Default  12813.000000             Trump  12813.000000  29.0000  29.0000
548   Default  11460.000000           Prozent  11460.000000  28.0000  28.0000
1119  Default   5140.000000          Russland   5140.000000  27.0000  27.0000
866   Default   6573.000000           Polizei   6573.000000  26.0000  26.0000
305   Default   7007.000000           Million   7007.000000  25.0000  25.0000
885   Default   4535.000000             Stadt   4535.000000  24.0000  24.0000
191   Default   4009.000000              Teil   4009.000000  23.0000  23.0000
897   Default   5185.000000              Zeit   5185.000000  22.0000  22.0000
142   Default   5853.000000            Dollar   5853.000000  21.0000  21.0000
57    Default   5391.000000         Regierung   5391.000000  20.0000  20.0000
118   Default   5406.000000               Tag   5406.000000  19.0000  19.0000
2276  Default   3741.000000              Buch   3741.000000  18.0000  18.0000
294   Default   3223.000000             Daten   3223.000000  17.0000  17.0000
1874  Default   3990.000000                EU   3990.000000  16.0000  16.0000
352   Default   3792.000000          Dienstag   3792.000000  15.0000  15.0000
342   Default   4758.000000  vereinigen_Staat   4758.000000  14.0000  14.0000
13    Default   8117.000000       Deutschland   8117.000000  13.0000  13.0000
1775  Default   3783.000000              Film   3783.000000  12.0000  12.0000
464   Default   1910.000000              Arzt   1910.000000  11.0000  11.0000
709   Default   2176.000000       Möglichkeit   2176.000000  10.0000  10.0000
614   Default   4006.000000              Bild   4006.000000   9.0000   9.0000
415   Default   2962.000000            Syrien   2962.000000   8.0000   8.0000
3603  Default   3020.000000               AfD   3020.000000   7.0000   7.0000
356   Default   9362.000000              Euro   9362.000000   6.0000   6.0000
929   Default   3227.000000               CSU   3227.000000   5.0000   5.0000
945   Default   3179.000000             Vater   3179.000000   4.0000   4.0000
251   Default   2747.000000         Politiker   2747.000000   3.0000   3.0000
778   Default   4381.000000             Monat   4381.000000   2.0000   2.0000
815   Default   3127.000000              Ziel   3127.000000   1.0000   1.0000
...       ...           ...               ...           ...      ...      ...
272   Topic10    173.239609         Verfügung   1381.294067   0.3235  -8.0354
342   Topic10    499.527710  vereinigen_Staat   4758.621094   0.1456  -6.9764
310   Topic10    289.140320            Nutzer   2515.357422   0.2364  -7.5231
651   Topic10    392.169495             Seite   3641.227295   0.1713  -7.2183
363   Topic10    259.480286         Interview   2221.974609   0.2522  -7.6313
1055  Topic10    377.182281              Wahl   3493.023193   0.1739  -7.2573
116   Topic10    352.622864            Studie   3238.051514   0.1823  -7.3246
1379  Topic10    308.256134   Bundesregierung   2746.805420   0.2124  -7.4591
914   Topic10    653.486572              Kind   7225.473633  -0.0034  -6.7077
897   Topic10    505.386749      

### Evaluation